# This example notebook shows how to create a source catalog through the JWST pipeline source_catalog step

The specifics are done for a NIRCAM Grism direct image, since the catalog is needed to define the source locations in the grism observation so that the spectral bounding boxes for each order can be extracted for each object. 

In [ ]:
%matplotlib inline
import os
import jwst
import numpy as np
from astropy.io import fits
print("Using jwst pipeline version: {}".format(jwst.__version__))

### I'm going to use a simulated image as an example, the cell below details its file structure
**Make sure that you have set the JWST_NOTEBOOK_DATA environment variable in the terminal from which you started Jupyter Notebook.**

The data will be read from that directory, and the pipeline should write to the current working directory, avoiding clobbers. 

In [ ]:
data_dir = os.environ['JWST_NOTEBOOK_DATA']
nircam_data= data_dir + 'data/nircam/'
grism_direct_image = nircam_data + 'nircam_grism_direct_image.fits'
fits.info(grism_direct_image)

### Let's take a visual look at the image we are using

In [ ]:
from matplotlib import pyplot as plt
dirim = fits.getdata(grism_direct_image, ext=1)
xs,ys=dirim.shape
fig = plt.figure(figsize=(4,4), dpi=150)
ax = fig.add_subplot(1, 1, 1)
ax.set_adjustable('box-forced')
ax.set_title(grism_direct_image.split("/")[-1], fontsize=8)
ax.imshow(dirim, origin='lower', extent=[0,xs,0,ys], vmin=-10, vmax=10)

### Import the catalog creation step from the pipeline

In [ ]:
from jwst.source_catalog import source_catalog_step

### Create the step object, it knows how to create the catalog when it's given a data model

In [ ]:
sc=source_catalog_step.SourceCatalogStep()

### These are the default step parameters

In [ ]:
print(sc.spec)

### This step only works on Drizzle products and single drizzle FITS images
You can open the fits image into a datamodel or you can supply the step with the name of the FITS image directly

In [ ]:
# Here we'll use a DataModel
from jwst.datamodels import DrizProductModel
dpm=DrizProductModel(grism_direct_image)

In [ ]:
sc.process(dpm)

In [ ]:
from astropy.table import QTable
source_catalog = 'nircam_grism_direct_image_cat.ecsv'  # this name is listed in the output of the process above

### The INFO log suggests that's too many sources, let's see how we did
The catalog cat be read is as an astropy quantities table

In [ ]:
catalog = QTable.read(source_catalog,  format='ascii.ecsv')

In [ ]:
catalog

### Overplot the object detections on our image

In [ ]:
import matplotlib.patches as patches
dirim = fits.getdata(grism_direct_image, ext=1)
xs,ys=dirim.shape
fig = plt.figure(figsize=(4,4), dpi=150)
ax.ticklabel_format(useOffset=False)
ax = fig.add_subplot(1, 1, 1)
ax.ticklabel_format(useOffset=False)
ax.set_adjustable('box-forced')
ax.set_title(grism_direct_image.split("/")[-1], fontsize=8)
ax.imshow(dirim, origin='lower', extent=[0,xs,0,ys], vmin=-10, vmax=10)

# rectangle patches are xmin, ymin, width, height
plist1=[]
for obj in catalog:
    plist1.append(patches.Circle((obj['xcentroid'].value, obj['ycentroid'].value),10))

for p in plist1:
    ax.add_patch(p)
    
ax.imshow(dirim, origin='lower', extent=[0,xs,0,ys], vmin=-10, vmax=10)

### Looks like we should edit the defaults so that we can restrict the detections to the visible objects

In [ ]:
# as a reminder, these are the defaults
print(sc.spec)

### The defaults can be changes in the step object directly

In [ ]:
sc.npixels=20
sc.snr_threshold=5
sc(dpm)

In [ ]:
# Read the new table that was created
catalog = QTable.read(source_catalog,  format='ascii.ecsv')
dirim = fits.getdata(grism_direct_image, ext=1)
xs,ys=dirim.shape
fig = plt.figure(figsize=(4,4), dpi=150)
ax.ticklabel_format(useOffset=False)
ax = fig.add_subplot(1, 1, 1)
ax.ticklabel_format(useOffset=False)
ax.set_adjustable('box-forced')
ax.set_title(grism_direct_image.split("/")[-1], fontsize=8)
ax.imshow(dirim, origin='lower', extent=[0,xs,0,ys], vmin=-10, vmax=10)

# rectangle patches are xmin, ymin, width, height
plist1=[]
for obj in catalog:
    plist1.append(patches.Circle((obj['xcentroid'].value, obj['ycentroid'].value),10))

for p in plist1:
    ax.add_patch(p)
    
ax.imshow(dirim, origin='lower', extent=[0,xs,0,ys], vmin=-10, vmax=10)

### Not bad!  There were two nearby sources that were not deblended, but we'll leave that for now.

In [ ]:
# checkout the final catalog
print(source_catalog)
catalog